## MPO HGA Blockage

This is a Python Jupyter Notebook to illustrate how to visualize the blockage that suffers the High Gain Antenna due to the MPO structure. 

First thing we will do is indicate that we want the Python package matplotlib to be output in the notebook and to import the SpiceyPy package to use SPICE. We also ensure that project_path module is include, this will allow us get access to common packages and resources

In [ ]:
from project_path import data_path
import spiceypy as cspice
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import trange

We will setup the kernel set for the spice library, that will be used along the notebook. Note that if you are in **Datalabs** infrastructure and the SPICE volume is mounted, the latest kernel repositories are available. Additionally the *ess.datalabs* package contains many helpers and shortcuts.

Using the *data_path* shortcut we can get resources stored in the **ess-jupyternb/data** folder

In [ ]:
from ess.datalabs import get_local_metakernel, is_spice_volume_mounted

cspice.kclear()
if is_spice_volume_mounted():
    cspice.furnsh(get_local_metakernel('BEPICOLOMBO', 'bc_plan.tm'))
else:
   cspice.furnsh('/Users/randres/git/spice/bepicolombo/kernels/mk/bc_plan.tm')

cspice.furnsh(
    os.path.join(data_path, 'test', 'bc_mpo_hga_schulte_vector_test_v01.bc'))
cspice.furnsh(
    os.path.join(data_path, 'test', 'bc_mpo_sc_bus_v03.bds'))

The *bc_mpo_hga_schulte_vector_test_v01.bc* contains a full rotation of the schulte vector. It contains the attitude covering in 3600 seconds the full excursion along 360 degrees. The start point in 2021-01-01T00:00:00Z

In [ ]:
time_step_seconds = 10 # time step
et0 = cspice.utc2et('2021-01-01T00:00:00')
et = et0
rotation_vectors = []
for j in range(-180,181):
    M = cspice.pxform('MPO_HGA', 'MPO_SPACECRAFT', et)
    rotation_vectors.append(cspice.mxv(M, [0, 0, 1])) 
    et += time_step_seconds
rv = np.array(rotation_vectors)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(rv[:,0],rv[:,1],rv[:,2]);
cspice.et2utc(et, 'ISOC', 0)


Now we calculate the blockage map with the help of this schulte vector. We simulate the exclusion for the dish of the antenna.

**Warning** It can take some minutes to get the map

In [ ]:

time_step_seconds = 10 # time step
angular_step_degrees = 12 
et0 = cspice.utc2et('2021-01-01T00:00:00')
el_angles = np.arange(-5.0, 145, 1)
az_angles = np.arange(-180, 181, 1)
coverage_map = np.zeros((el_angles.shape[0], az_angles.shape[0]))
et = et0
output = []
output_b = []
row_counter = el_angles.shape[0]
for i in trange(el_angles.shape[0]):
    for j in range(az_angles.shape[0]):
        el = el_angles[i]
        az = az_angles[j]
        rs = []
        ds = []
        for n in range(0, 360, angular_step_degrees):
            n = np.deg2rad(n)
            r0 = np.array([np.cos(n), np.sin(n), 0]) * 0.0005
            r_opt = cspice.spkpos('MPO_HGA_OPT_EL', et, 'MPO_SPACECRAFT', 'NONE', 'MPO_SPACECRAFT')[0]
            M = cspice.pxform('MPO_HGA', 'MPO_SPACECRAFT', et)
            r = cspice.mxv(M, r0) + r_opt
            d = cspice.mxv(M, [0, 0, 1])
            rs.append(r)
            ds.append(d)
            
        xarray, flag_array = cspice.dskxv(False, 'MPO_SPACECRAFT', [-121000], et, 'MPO_SPACECRAFT',
                         rs, ds)
        
        coverage_map[i, j] += len(list(filter(lambda x: x, flag_array)))           
        et += time_step_seconds



We can now visualize the colored blockage map

In [ ]:
plt.contourf(range(-180, 181), range(-5, 145), coverage_map, 100)
plt.ylim([90, 144])
plt.colorbar()
plt.grid()
plt.show()

In [ ]:
coverage = coverage_map / (360 / angular_step_degrees) * 100
coverage[coverage == 0] = np.nan

fig = plt.figure(figsize=(14, 6))
plt.contourf(range(-180, 181), range(-5, 145), coverage, 20, alpha=0.1)
contour_lines = plt.contour(range(-180, 181), range(-5, 145), coverage, 20)
plt.clabel(contour_lines, contour_lines.levels, inline=True, fmt='%1.0f', fontsize=8)
plt.colorbar()
plt.ylim([90, 144])
plt.grid()
plt.ylabel('El [deg]')
plt.xlabel('Az [deg]')
plt.title('HGA disk percentage blockage')
plt.show()